<a href="https://colab.research.google.com/github/catch-twenty2/AstroChart_Analysis/blob/main/AstroChart_Analysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run this cell the first time only to install core packages
!pip install pyswisseph geopy ipywidgets timezonefinder

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [3]:
import swisseph as swe
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import contextlib
from zoneinfo import ZoneInfo
from timezonefinder import TimezoneFinder

def get_timezone(lat, lon, date):
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=lat, lng=lon)  # Changed from certain_timezone_at to timezone_at and lon to lng
    if timezone_str is None:
        return ZoneInfo("UTC")
    return ZoneInfo(timezone_str)


def is_dst(dt, timezone):
    return timezone.dst(dt) != timedelta(0)

def calculate_chart(year, month, day, hour, minute, lat, lon):
    # Get the timezone for the given location
    local_tz = get_timezone(lat, lon, datetime(year, month, day))

    # Create a datetime object in the local timezone
    local_dt = datetime(year, month, day, hour, minute, tzinfo=local_tz)

    # Convert to UTC
    utc_dt = local_dt.astimezone(ZoneInfo("UTC"))

    # Set the Julian day using UTC time
    jd = swe.julday(utc_dt.year, utc_dt.month, utc_dt.day, utc_dt.hour + utc_dt.minute / 60.0)

    # Set geographical location
    swe.set_topo(lon, lat, 0)

    # List of planets to calculate
    planets = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto']
    planet_ids = [swe.SUN, swe.MOON, swe.MERCURY, swe.VENUS, swe.MARS, swe.JUPITER, swe.SATURN, swe.URANUS, swe.NEPTUNE, swe.PLUTO]

    results = {}

    # Calculate Ascendant and MC
    houses = swe.houses(jd, lat, lon, b'P')[0]  # Placidus house system
    ascendant = houses[0]
    mc = houses[9]

    # Calculate planet positions
    for planet, planet_id in zip(planets, planet_ids):
        position = swe.calc_ut(jd, planet_id, swe.FLG_SWIEPH)[0]
        longitude = position[0]
        results[planet] = {
            'longitude': longitude,
            'sign': get_sign(longitude),
            'house': get_house(longitude, houses)
        }

    return results, ascendant, mc

def get_sign(longitude):
    signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo',
             'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']
    return signs[int(longitude / 30)]

def get_house(longitude, houses):
    for i in range(12):
        if longitude >= houses[i] and longitude < houses[(i+1) % 12]:
            return i + 1
    return 12  # Default to 12th house if not found

def calculate_aspects(positions, orb=5):
    aspects = []
    aspect_types = {
        0: 'Conjunction',
        60: 'Sextile',
        90: 'Square',
        120: 'Trine',
        180: 'Opposition'
    }

    planets = list(positions.keys())
    for i, p1 in enumerate(planets):
        for p2 in planets[i+1:]:
            diff = abs(positions[p1]['longitude'] - positions[p2]['longitude'])
            diff = min(diff, 360 - diff)
            for angle, aspect_name in aspect_types.items():
                if abs(diff - angle) <= orb:
                    aspects.append(f"{p1} {aspect_name} {p2} (orb: {abs(diff - angle):.2f})")
    return aspects

# Input widgets for user name, birth and current location information
user_name_input = widgets.Text(
    value='Neo',
    placeholder='Enter your name',
    description='Name:',
    disabled=False
)

birth_date_input = widgets.Text(
    value='2003-04-29',
    placeholder='YYYY-MM-DD',
    description='Birth Date:',
    disabled=False
)

birth_time_input = widgets.Text(
    value='12:00',
    placeholder='HH:MM',
    description='Birth Time (if unknown leave the default):',
    disabled=False
)

birth_lat_input = widgets.FloatText(
    value=52.52,
    placeholder='Enter birth latitude',
    description='Birth Lat:',
    disabled=False
)

birth_lon_input = widgets.FloatText(
    value=13.40,
    placeholder='Enter birth longitude',
    description='Birth Lon:',
    disabled=False
)

current_lat_input = widgets.FloatText(
    value=52.52,
    placeholder='Enter current latitude',
    description='Current Lat:',
    disabled=False
)

current_lon_input = widgets.FloatText(
    value=13.40,
    placeholder='Enter current longitude',
    description='Current Lon:',
    disabled=False
)

target_date_input = widgets.Text(
    value='2024-09-01',
    placeholder='YYYY-MM-DD',
    description='Target Date:',
    disabled=False
)

button = widgets.Button(
    description='Calculate Charts',
    disabled=False,
    button_style='',
    tooltip='Click to calculate charts',
    icon='check'
)

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        user_name = user_name_input.value
        birth_date = birth_date_input.value
        birth_time = birth_time_input.value
        birth_lat = birth_lat_input.value
        birth_lon = birth_lon_input.value
        current_lat = current_lat_input.value
        current_lon = current_lon_input.value
        target_date = target_date_input.value

        year, month, day = map(int, birth_date.split('-'))
        hour, minute = map(int, birth_time.split(':'))

        # Get the timezone for the birth location
        birth_tz = get_timezone(birth_lat, birth_lon, datetime(year, month, day))

        # Create a datetime object in the birth timezone
        birth_dt = datetime(year, month, day, hour, minute, tzinfo=birth_tz)

        # Check if it's DST
        if is_dst(birth_dt, birth_tz):
            print(f"Note: The birth time is during Daylight Saving Time in the {birth_tz} timezone.")

        # Capture output for natal chart
        natal_output = io.StringIO()
        with contextlib.redirect_stdout(natal_output):
            # Calculate natal chart
            natal_positions, ascendant, mc = calculate_chart(year, month, day, hour, minute, birth_lat, birth_lon)
            print("Natal Chart Positions:")
            for planet, data in natal_positions.items():
                print(f"{planet}: {data}")
            print(f"\nAscendant: {get_sign(ascendant)} ({ascendant:.2f}°)")
            print(f"Midheaven: {get_sign(mc)} ({mc:.2f}°)")

            aspects = calculate_aspects(natal_positions)
            print("\nAspects:")
            for aspect in aspects:
                print(aspect)
            print()

        natal_output = natal_output.getvalue()

        # Capture output for relocation chart if current location is provided
        relocation_output = ""
        if current_lat != 0.0 and current_lon != 0.0:
            # Capture output for relocation chart
            relocation_output = io.StringIO()
            with contextlib.redirect_stdout(relocation_output):
                # Calculate relocation chart for the target date
                target_year, target_month, target_day = map(int, target_date.split('-'))
                relocation_positions, ascendant, mc = calculate_chart(target_year, target_month, target_day, hour, minute, current_lat, current_lon)
                print("Relocation Chart Positions for Target Date:")
                for planet, data in relocation_positions.items():
                    print(f"{planet}: {data}")
                print(f"\nAscendant: {get_sign(ascendant)} ({ascendant:.2f}°)")
                print(f"Midheaven: {get_sign(mc)} ({mc:.2f}°)")

                aspects = calculate_aspects(relocation_positions)
                print("\nAspects:")
                for aspect in aspects:
                    print(aspect)
                print()

            relocation_output = relocation_output.getvalue()

        # Display the context for LLM
        context = f"""PROMPT FOR THE AI ASSISTANT
========================
Dear Assistant, act as a professional Astrologer. I have gathered detailed astrological data for a person, including both their natal chart and a relocation chart for a specific target date. I would like a professional astrological interpretation of this data. Below is the information provided:

Context:
Name: {user_name}
Birth Date and Time: {birth_date}, {birth_time}
Birth Location: Lat: {birth_lat}, Lon: {birth_lon}
Current Location: Lat: {current_lat}, Lon: {current_lon}
Target Date: {target_date}

Please provide a professional interpretation of this data, focusing on the following aspects:

{natal_output}

{relocation_output}

Natal Chart Interpretation: Explain the inherent personality traits, life patterns, and potential challenges based on the natal chart data.
Relocation Chart Interpretation: Describe how the individual might experience life in the new location at the specified target date, highlighting any significant changes or continuities from the natal chart.
Aspect Analysis: Provide insights into the key themes and dynamics in the person's life as indicated by the aspects in both charts.
Overall Comparison: Summarize how the energies and influences in the natal chart compare to those in the relocation chart, and what this might mean for the individual's experiences and opportunities during the target period.

Suggest to user any followup questions dealing with challenges, and suggestions for how to bridge the gap in problematic issues.
Thank you!
"""

        print(context)

button.on_click(on_button_click)

# Display the widgets
display(user_name_input, birth_date_input, birth_time_input, birth_lat_input, birth_lon_input, current_lat_input, current_lon_input, target_date_input, button, output)

Text(value='Neo', description='Name:', placeholder='Enter your name')

Text(value='2003-04-29', description='Birth Date:', placeholder='YYYY-MM-DD')

Text(value='12:00', description='Birth Time (if unknown leave the default):', placeholder='HH:MM')

FloatText(value=52.52, description='Birth Lat:')

FloatText(value=13.4, description='Birth Lon:')

FloatText(value=52.52, description='Current Lat:')

FloatText(value=13.4, description='Current Lon:')

Text(value='2024-09-01', description='Target Date:', placeholder='YYYY-MM-DD')

Button(description='Calculate Charts', icon='check', style=ButtonStyle(), tooltip='Click to calculate charts')

Output()